In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os




In [ ]:
test_extract_path='/kaggle/input/rhdatasetv1/test/'
train_extract_path='/kaggle/input/rhdatasetv1/train/'

# List the contents of the extracted folders
print("Contents of train folder:")
print(os.listdir(os.path.join(train_extract_path, 'train')))

print("\nContents of test folder:")
print(os.listdir(os.path.join(test_extract_path, 'test')))

In [ ]:
# Load CSV files
dataset_path='/kaggle/input/rhdatasetv1/'
train_csv_path = os.path.join(dataset_path, 'train/train/train.csv')
test_csv_path = os.path.join(dataset_path, 'test/test/test.csv')

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)


train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.isna().sum()

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import seaborn as sns
sns.countplot(data=train_df,x='label')

In [ ]:
train_df.label.value_counts(normalize=True)*100

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2,EfficientNetB7
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score
from keras.utils import to_categorical

num_classes=6
img_size=600

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convert 'label' column to string
train_df['label'] = train_df['label'].astype(str)

# # Use LabelEncoder
# label_encoder = LabelEncoder()
# train_df['label'] = label_encoder.fit_transform(train_df['label'])

# # Convert labels to one-hot encoding
# train_df['label'] = to_categorical(train_df['label'])

dataset_path = '/kaggle/input/rhdatasetv1/'

# Split the dataset into training and validation sets
train_data, valid_data = train_test_split(train_df, test_size=0.2, random_state=42, shuffle=True, stratify=train_df.label)

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=os.path.join(dataset_path, 'train/train/images'),
    x_col='filename',
    y_col='label',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = val_datagen.flow_from_dataframe(
    dataframe=valid_data,
    directory=os.path.join(dataset_path, 'train/train/images'),
    x_col='filename',
    y_col='label',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
valid_data.shape,train_data.shape


In [ ]:
import cv2
import numpy as np
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB7
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os


NUM_CLASSES = 6
IMG_SIZE = 224

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = inputs
    model = tf.keras.applications.EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model



In [ ]:

model = build_model(num_classes=NUM_CLASSES)  # Change num_classes accordingly

# Define callbacks
checkpoint_callback = ModelCheckpoint("model_checkpoint.h5", save_best_only=True)
early_stopping_callback = EarlyStopping(patience=3, restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir="logs_effi")

#Train the model using the data generators
history = model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=valid_generator,
    callbacks=[checkpoint_callback, early_stopping_callback, tensorboard_callback],
    verbose=1
)

In [ ]:
ls /kaggle/input/rhdatasetv1/test/test/


In [ ]:
pwd

In [ ]:
import pandas as pd
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Set the path to your dataset folder
data_test_path = '/kaggle/input/rhdatasetv1/test/test/'

# Load the trained model
model_path = '/kaggle/working/model_checkpoint.h5'
trained_model = load_model(model_path)

# Load the test data CSV
test_csv_path = os.path.join(data_test_path, 'test.csv')
test_df = pd.read_csv(test_csv_path)

# Set the path to the test images folder
test_images_path = os.path.join(data_test_path, 'images')

# Create a list to store DataFrames
result_list = []

# Batch processing
batch_size = 32  # Adjust based on your available memory
for i in range(0, len(test_df), batch_size):
    batch_df = test_df.iloc[i:i + batch_size]
    batch_images = []

    for _, row in batch_df.iterrows():
        image_path = os.path.join(test_images_path, row['filename'])
        img = image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        batch_images.append(img_array)

    batch_images = np.vstack(batch_images)  # Stack the images to create a batch
    predictions = trained_model.predict(batch_images)

    # Create a DataFrame for each batch
    batch_result_df = pd.DataFrame({
        'image_id': batch_df['image_id'],
        'label': np.argmax(predictions, axis=1)
    })

    result_list.append(batch_result_df)

# Concatenate the list of DataFrames
result_df = pd.concat(result_list, ignore_index=True)

# Save the predictions to a new CSV file with the 'label' column
submission_csv_path = '/kaggle/working/submission_Eb01.csv'
result_df.to_csv(submission_csv_path, index=False)

print(f"Predictions saved to: {submission_csv_path}")

In [ ]:
# # Build a CNN model using MobileNetV2 as the base
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# model = models.Sequential()
# model.add(base_model)
# model.add(layers.GlobalAveragePooling2D())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(6, activation='softmax'))  # Output layer with 6 classes

# # Compile the model
# model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
pwd

In [ ]:
ls

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# Load the contents of submission.csv
submission_df = pd.read_csv('/kaggle/working/submission_Eb01.csv')

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title="Download CSV file", filename="submission_Eb01.csv"):
    csv = df.to_csv(index=False)  # Avoid writing the index to the CSV
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload, title=title, filename=filename)
    return HTML(html)

# Create a link to download the dataframe
create_download_link(submission_df)


In [ ]:
pwd

In [ ]:
ls 

In [ ]:
cd ..

In [ ]:
cd /kaggle/input

In [ ]:
ls

In [ ]:
cd rhdatasetv1/